In [1]:
import pandas as pd
import numpy as np
seed = np.random.seed(42)
from matplotlib import pyplot as plt
import seaborn as sns
sns.set(style="ticks", color_codes=True)

pd.set_option('display.max_columns',500)
pd.set_option('display.max_rows',50)

%matplotlib inline

In [2]:
data = pd.read_csv('../data/hospital' + str(1) + '_data.csv')

In [3]:
from sklearn.preprocessing import Imputer,StandardScaler,LabelEncoder
from fancyimpute import BiScaler, KNN, NuclearNormMinimization, SoftImpute, MICE

/Users/refikcanmalli/anaconda3/envs/vodafone/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [4]:
data_e = data[data['IPG'] != 1]
data_e = data_e.drop('IPG', axis = 1);
data_e = data_e.drop('sbjBeatConsidered', axis = 1);
data_e = data_e.drop('numRRaveraged', axis = 1);
data_e = data_e.drop('Soggetti', axis = 1);
df_columns = data.columns.tolist()
matching_columns1 = [s for s in df_columns if "Parea" in s]
matching_columns2 = [s for s in df_columns if "Pdur" in s]
mc = matching_columns1 + matching_columns2
data_e = data_e.dropna(subset=mc)
data_e['PCneg'].fillna(0, inplace=True)
data_e['AFclass'].replace('persistierend (>7 Tage, EKV)', 1, inplace=True)
data_e['AFclass'].replace('paroxysmal', 0, inplace=True)
categorical_variables_mask = data_e.dtypes==object
categorical_variables = data_e.columns[categorical_variables_mask]
for v in categorical_variables:
    data_e[v] = LabelEncoder().fit_transform(data_e[v])
del categorical_variables_mask,categorical_variables
data_e = pd.DataFrame(data=MICE(n_imputations=100, impute_type='col', n_nearest_columns=5).complete(data_e.as_matrix()), columns=data_e.columns, index=data_e.index);
y_df = data_e['AFclass'].copy()
x_df = data_e.drop('AFclass',axis=1, inplace=False)

[MICE] Completing matrix with shape (247, 69)
[MICE] Starting imputation round 1/110, elapsed time 0.001
[MICE] Starting imputation round 2/110, elapsed time 0.014
[MICE] Starting imputation round 3/110, elapsed time 0.023
[MICE] Starting imputation round 4/110, elapsed time 0.031
[MICE] Starting imputation round 5/110, elapsed time 0.040
[MICE] Starting imputation round 6/110, elapsed time 0.048
[MICE] Starting imputation round 7/110, elapsed time 0.057
[MICE] Starting imputation round 8/110, elapsed time 0.065
[MICE] Starting imputation round 9/110, elapsed time 0.074
[MICE] Starting imputation round 10/110, elapsed time 0.082
[MICE] Starting imputation round 11/110, elapsed time 0.091
[MICE] Starting imputation round 12/110, elapsed time 0.099
[MICE] Starting imputation round 13/110, elapsed time 0.108
[MICE] Starting imputation round 14/110, elapsed time 0.116
[MICE] Starting imputation round 15/110, elapsed time 0.125
[MICE] Starting imputation round 16/110, elapsed time 0.133
[MI

In [6]:
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier,ExtraTreesClassifier,AdaBoostClassifier
#from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, make_scorer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.dummy import DummyClassifier
from imblearn.over_sampling import SMOTE
rom imblearn.pipeline import Pipeline

#f1_scorer = make_scorer(f1_score, average='macro',labels=Y.unique().tolist())
#myscorer = f1_scorer
myscorer = 'roc_auc'

In [7]:
balancer = SMOTE(random_state=42)

In [8]:
def show_f_importance(estimator,train_data):
    importances = estimator.feature_importances_
    indices = np.argsort(importances)[::-1]
    indices = np.argsort(importances)[::-1]
    # Print the feature ranking
    print("Feature ranking:")

    for f in range(train_data.shape[1]):
        print("%d. feature %d - %s (%f)" % (f + 1, indices[f],train_data.columns[indices[f]], importances[indices[f]]))


In [11]:
param_grid = {"clf__strategy": ['most_frequent','stratified','prior','uniform']}

dummy = DummyClassifier(random_state=42)
plist = []
plist.append(('balancer', balancer))
plist.append(('clf', dummy))
pipeline = Pipeline(plist)

gs_dummy = GridSearchCV(pipeline, param_grid=param_grid, cv=5, verbose=1, scoring=myscorer, n_jobs=-1)

TypeError: All intermediate steps should be transformers and implement fit and transform. 'SMOTE(k=None, k_neighbors=5, kind='regular', m=None, m_neighbors=10, n_jobs=1,
   out_step=0.5, random_state=42, ratio='auto', svm_estimator=None)' (type <class 'imblearn.over_sampling.smote.SMOTE'>) doesn't

In [8]:
gs_dummy.fit(x_df, y_df);

Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:    0.1s finished


In [9]:
gs_dummy.best_params_

{'strategy': 'stratified'}

In [10]:
gs_dummy.best_score_

0.5918292234081708

In [11]:
param_grid = {"max_depth": [10,30,50],
              "criterion": ['gini', 'entropy'],
              "n_estimators": [100,200,500,1000],
              "random_state": [42],
              "class_weight" : ['balanced']}
# param_grid = {"classifier__max_depth": [30],
#               "classifier__n_estimators": [1000]}

rf = RandomForestClassifier()
gs_rf = GridSearchCV(rf, param_grid=param_grid, cv=5, verbose=1, scoring=myscorer, n_jobs=-1)

In [12]:
gs_rf.fit(x_df, y_df);

Fitting 5 folds for each of 24 candidates, totalling 120 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   14.4s
[Parallel(n_jobs=-1)]: Done 120 out of 120 | elapsed:  3.0min finished


In [13]:
gs_rf.best_params_

{'class_weight': 'balanced',
 'criterion': 'entropy',
 'max_depth': 30,
 'n_estimators': 200,
 'random_state': 42}

In [14]:
gs_rf.best_score_

0.5969876034698016

In [15]:
show_f_importance(gs_rf.best_estimator_,x_df)

Feature ranking:
1. feature 53 - Pfwhm_ECG_V1 (0.030746)
2. feature 46 - Parea_ECG_V9 (0.027979)
3. feature 39 - Parea_ECG_V3 (0.027032)
4. feature 45 - Parea_ECG_V8 (0.023360)
5. feature 16 - Pdur16std (0.022333)
6. feature 3 - QTn (0.021783)
7. feature 56 - Pfwhm_ECG_V3R (0.021477)
8. feature 0 - RR (0.020819)
9. feature 1 - QTm_old (0.020461)
10. feature 36 - Parea_ECG_III (0.020421)
11. feature 55 - Pfwhm_ECG_V3 (0.020086)
12. feature 40 - Parea_ECG_V3R (0.019993)
13. feature 48 - Parea_ECG_aVL (0.019189)
14. feature 54 - Pfwhm_ECG_V2 (0.019108)
15. feature 41 - Parea_ECG_V4 (0.018331)
16. feature 57 - Pfwhm_ECG_V4 (0.017986)
17. feature 10 - PCarea (0.017047)
18. feature 44 - Parea_ECG_V6 (0.016879)
19. feature 42 - Parea_ECG_V4R (0.016751)
20. feature 43 - Parea_ECG_V5 (0.016527)
21. feature 67 - Age (0.016420)
22. feature 5 - PQ (0.016112)
23. feature 4 - QRS (0.016012)
24. feature 35 - Parea_ECG_II (0.015933)
25. feature 34 - Parea_ECG_I (0.015681)
26. feature 9 - PCfwhm (0.015

In [16]:
param_grid = {"max_depth": [3,5,8],
              "n_estimators": [100,200,500,1000],
              "min_samples_leaf": [1,3,5,10]}
# param_grid = {"classifier__n_estimators": [50],
#               "classifier__max_depth": [5],
#            

gb = GradientBoostingClassifier(random_state=42)
gs_gb = GridSearchCV(gb, param_grid=param_grid, cv=5, verbose=1,  scoring=myscorer, n_jobs=-1)

In [17]:
gs_gb.fit(x_df, y_df);

Fitting 5 folds for each of 48 candidates, totalling 240 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   44.0s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done 240 out of 240 | elapsed:  3.4min finished


In [18]:
gs_gb.best_params_

{'max_depth': 3, 'min_samples_leaf': 5, 'n_estimators': 500}

In [19]:
gs_gb.best_score_

0.558107269407579

In [20]:
show_f_importance(gs_gb.best_estimator_,x_df)

Feature ranking:
1. feature 1 - QTm_old (0.040443)
2. feature 67 - Age (0.036442)
3. feature 46 - Parea_ECG_V9 (0.032061)
4. feature 40 - Parea_ECG_V3R (0.031489)
5. feature 16 - Pdur16std (0.031477)
6. feature 53 - Pfwhm_ECG_V1 (0.030074)
7. feature 2 - QR (0.029743)
8. feature 54 - Pfwhm_ECG_V2 (0.029142)
9. feature 0 - RR (0.027692)
10. feature 64 - Pfwhm_ECG_aVL (0.026485)
11. feature 43 - Parea_ECG_V5 (0.026139)
12. feature 48 - Parea_ECG_aVL (0.024880)
13. feature 21 - Pdur_ECG_V1 (0.023731)
14. feature 39 - Parea_ECG_V3 (0.021949)
15. feature 56 - Pfwhm_ECG_V3R (0.021740)
16. feature 6 - PCpos (0.021155)
17. feature 62 - Pfwhm_ECG_V9 (0.020805)
18. feature 61 - Pfwhm_ECG_V8 (0.020418)
19. feature 17 - Pdur16iqr (0.019120)
20. feature 51 - Pfwhm_ECG_II (0.018844)
21. feature 41 - Parea_ECG_V4 (0.018803)
22. feature 52 - Pfwhm_ECG_III (0.017133)
23. feature 19 - Pdur_ECG_II (0.016074)
24. feature 3 - QTn (0.015870)
25. feature 33 - Pdur_ECG_aVR (0.015773)
26. feature 30 - Pdur_ECG

In [21]:
param_grid = {"learning_rate": [0.01,0.1,0.3],
              "n_estimators": [100,1000,1500,2000,2500,3000]}

ab = AdaBoostClassifier(random_state=42)
gs_ab = GridSearchCV(ab, param_grid=param_grid, cv=5, verbose=1, scoring=myscorer, n_jobs=-1)

In [22]:
gs_ab.fit(x_df, y_df);

Fitting 5 folds for each of 18 candidates, totalling 90 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  3.5min
[Parallel(n_jobs=-1)]: Done  90 out of  90 | elapsed:  9.2min finished


In [23]:
gs_ab.best_params_

{'learning_rate': 0.1, 'n_estimators': 3000}

In [24]:
gs_ab.best_score_

0.5904251192563886

In [25]:
show_f_importance(gs_ab.best_estimator_,x_df)

Feature ranking:
1. feature 16 - Pdur16std (0.051000)
2. feature 39 - Parea_ECG_V3 (0.044000)
3. feature 6 - PCpos (0.039667)
4. feature 40 - Parea_ECG_V3R (0.037667)
5. feature 21 - Pdur_ECG_V1 (0.036667)
6. feature 19 - Pdur_ECG_II (0.032667)
7. feature 46 - Parea_ECG_V9 (0.030333)
8. feature 52 - Pfwhm_ECG_III (0.030333)
9. feature 38 - Parea_ECG_V2 (0.029667)
10. feature 54 - Pfwhm_ECG_V2 (0.028667)
11. feature 53 - Pfwhm_ECG_V1 (0.026333)
12. feature 3 - QTn (0.024667)
13. feature 63 - Pfwhm_ECG_aVF (0.024333)
14. feature 61 - Pfwhm_ECG_V8 (0.024333)
15. feature 17 - Pdur16iqr (0.024000)
16. feature 11 - P_LoffEon (0.023000)
17. feature 43 - Parea_ECG_V5 (0.023000)
18. feature 48 - Parea_ECG_aVL (0.022667)
19. feature 67 - Age (0.021333)
20. feature 2 - QR (0.021000)
21. feature 56 - Pfwhm_ECG_V3R (0.020667)
22. feature 57 - Pfwhm_ECG_V4 (0.020333)
23. feature 20 - Pdur_ECG_III (0.017667)
24. feature 33 - Pdur_ECG_aVR (0.017000)
25. feature 25 - Pdur_ECG_V4 (0.016667)
26. feature 

In [26]:
param_grid = {"max_depth": [3,5,8,10,15,30],
              "n_estimators": [100,200,500,1000,1500],
              "min_samples_leaf": [1,3,5,10]}

et = ExtraTreesClassifier(random_state=42)
gs_et = GridSearchCV(et, param_grid=param_grid, cv=5, verbose=1,  scoring=myscorer, n_jobs=-1)

In [27]:
gs_et.fit(x_df, y_df);

Fitting 5 folds for each of 120 candidates, totalling 600 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   14.5s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:  2.9min
[Parallel(n_jobs=-1)]: Done 600 out of 600 | elapsed:  3.9min finished


In [28]:
gs_et.best_params_

{'max_depth': 8, 'min_samples_leaf': 5, 'n_estimators': 1000}

In [29]:
gs_et.best_score_

0.6311996182352221

In [30]:
show_f_importance(gs_et.best_estimator_,x_df)

Feature ranking:
1. feature 66 - patsex (0.040856)
2. feature 46 - Parea_ECG_V9 (0.033252)
3. feature 45 - Parea_ECG_V8 (0.032705)
4. feature 16 - Pdur16std (0.029778)
5. feature 53 - Pfwhm_ECG_V1 (0.029042)
6. feature 39 - Parea_ECG_V3 (0.028678)
7. feature 67 - Age (0.024388)
8. feature 1 - QTm_old (0.021374)
9. feature 38 - Parea_ECG_V2 (0.020938)
10. feature 41 - Parea_ECG_V4 (0.019993)
11. feature 56 - Pfwhm_ECG_V3R (0.019915)
12. feature 17 - Pdur16iqr (0.018506)
13. feature 40 - Parea_ECG_V3R (0.018321)
14. feature 54 - Pfwhm_ECG_V2 (0.018054)
15. feature 35 - Parea_ECG_II (0.017170)
16. feature 44 - Parea_ECG_V6 (0.016979)
17. feature 48 - Parea_ECG_aVL (0.016960)
18. feature 3 - QTn (0.016822)
19. feature 43 - Parea_ECG_V5 (0.016207)
20. feature 10 - PCarea (0.016166)
21. feature 36 - Parea_ECG_III (0.016075)
22. feature 49 - Parea_ECG_aVR (0.015841)
23. feature 55 - Pfwhm_ECG_V3 (0.015611)
24. feature 59 - Pfwhm_ECG_V5 (0.014696)
25. feature 57 - Pfwhm_ECG_V4 (0.014307)
26. f